<a href="https://colab.research.google.com/github/AriannaHeartbell/sdproject/blob/main/upscaler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spandrel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.9/287.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.7 MB/s eta 0:00:00


In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


참조: https://chat.openai.com/share/ab2673a1-8a17-4551-a317-eeda61ec3654

In [ ]:
from torchvision.transforms import Resize
import os


# 폴더 경로 변수 설정
input_folder = '/content/drive/MyDrive/SD/UP/up_8'
output_folder = '/content/drive/MyDrive/SD/UP/upd_8'
model_path = '/content/drive/MyDrive/SD/models/RealESRGAN/RealESRGAN_x4plus_anime_6B.pth'

# 모델 로드
model = ModelLoader().load_from_file(model_path)
assert isinstance(model, ImageModelDescriptor)
model.cuda().eval()

# 이미지 처리 함수
def process(image: torch.Tensor) -> torch.Tensor:
    with torch.no_grad():
        return model(image)

# 이미지 크기 조절 함수
def resize_image(image, scale_factor=0.5):
    # 이미지를 RGBA로 변환 (투명도가 있는 팔레트 이미지인 경우)
    if image.mode == 'P':
        image = image.convert('RGBA')
    elif image.mode != 'RGB':
        image = image.convert('RGB')

    width, height = image.size
    new_width, new_height = int(width * scale_factor), int(height * scale_factor)
    resize = Resize((new_height, new_width))
    return resize(image)

# 하위 폴더를 포함한 모든 이미지 파일의 경로 찾기
image_paths = []
for subdir, dirs, files in os.walk(input_folder):
    for file in files:
        filepath = os.path.join(subdir, file)
        if filepath.endswith(".jpg") or filepath.endswith(".png"):
            image_paths.append(filepath)

# 각 이미지 처리
for image_path in image_paths:
    image = Image.open(image_path)
    resized_image = resize_image(image)  # 이미지 크기 조절
    image_tensor = TF.to_tensor(resized_image).unsqueeze(0).cuda()  # 변환된 이미지를 텐서로 변환하고 GPU로 보냄
    processed_tensor = process(image_tensor)  # 이미지 처리
    processed_image = TF.to_pil_image(processed_tensor.squeeze(0))  # 텐서를 PIL 이미지로 변환

    # 상대 경로를 이용해 저장 경로 설정
    relative_path = os.path.relpath(image_path, input_folder)
    save_path = os.path.join(output_folder, relative_path)
    os.makedirs(os.path.dirname(save_path), exist_ok=True)  # 필요한 경우 폴더 생성
    processed_image.save(save_path)


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
